In [2]:
from rdkit import Chem
from rdkit.Chem.Draw import IPythonConsole
from rdkit.Chem import Draw
from rdkit.Chem import PandasTools
import xlsxwriter
import pandas as pd
import os,sys

In [3]:
def remove_isotope(smiles1):
    mol = Chem.MolFromSmiles(smiles1)
    for atom in mol.GetAtoms():
        if atom.GetIsotope():
            atom.SetIsotope(0)
    return mol

In [4]:
def add_bridging_bonds(ligandCN, ligandNN):
    Ir = Chem.MolFromSmiles('[Ir+3]')
    combo1 = Chem.CombineMols(ligandCN, Ir)
    combo2 = Chem.CombineMols(ligandCN, combo1)
    combo = Chem.CombineMols(ligandNN, combo2)
    edcombo = Chem.EditableMol(combo)
    #find all CN atoms to be coordinated
    clist = []
    nlist = []
    for atom in combo.GetAtoms():
        if atom.GetIsotope()>0:
            if atom.GetSymbol()=='C':
                clist.append(atom.GetIdx())
            else:
                nlist.append(atom.GetIdx())
        if atom.GetSymbol()=='Ir':
            Ir_index = atom.GetIdx()
    for cindex in clist:
        edcombo.AddBond(cindex,Ir_index,order=Chem.rdchem.BondType.SINGLE)
    for nindex in nlist:
        edcombo.AddBond(nindex,Ir_index,order=Chem.rdchem.BondType.DATIVE)
    back = edcombo.GetMol()
    #remove isotope
    for atom in back.GetAtoms():
        if atom.GetIsotope():
            atom.SetIsotope(0)
    smiles = Chem.MolToSmiles(back)
    return smiles      

In [5]:
cwd = os.getcwd()

In [6]:
df = pd.read_csv(cwd+'/ref_cn_nn_smiles.csv')
df

,complex_i,CN_smiles,NN_smiles
0,1,FC1=CC(F)=C(C2=[15N]C=C(C(F)(F)F)C=C2)[13C]=C1,CC(C)(C)C1=CC=[15N]C(C2=[15N]C=CC(C(C)(C)C)=C2...
1,2,FC1=CC(F)=C(C2=[15N]C=CC(C(C)(C)C)=C2)[13C]=C1,CC(C)(C)C1=CC=[15N]C(C2=[15N]C=CC(C(C)(C)C)=C2...
2,3,FC1=CC(F)=C(C2=[15N]C=CC=C2)[13C]=C1,CC(C)(C)C1=CC=[15N]C(C2=[15N]C=CC(C(C)(C)C)=C2...
3,4,C1(C2=[15N]C=CC=C2)=CC=CC=[13C]1,CC(C)(C)C1=CC=[15N]C(C2=[15N]C=CC(C(C)(C)C)=C2...
4,5,C1(C2=[15N]C=CC=C2)=CC=CC=[13C]1,COC1=CC=[15N]C(C2=[15N]C=CC(OC)=C2)=C1
5,6,CC(C)(C)C1=CC=[13C]C(C2=[15N]C=CC(C(C)(C)C)=C2...,CC(C)(C)C1=CC=[15N]C(C2=[15N]C=CC(C(C)(C)C)=C2...
6,7,CC(C)(C)C1=CC=[13C]C(C2=[15N]C=CC(C(C)(C)C)=C2...,COC1=CC=[15N]C(C2=[15N]C=CC(OC)=C2)=C1


In [7]:
smiles_list = []
for i in range(len(df)):
    cn_smiles = df['CN_smiles'][i]
    nn_smiles = df['NN_smiles'][i]
    cn = Chem.MolFromSmiles(cn_smiles)
    nn = Chem.MolFromSmiles(nn_smiles)
    cluster_smiles = add_bridging_bonds(cn, nn)
    smiles_list.append(cluster_smiles)

In [8]:
df['complex_smiles'] = smiles_list
df

,complex_i,CN_smiles,NN_smiles,complex_smiles
0,1,FC1=CC(F)=C(C2=[15N]C=C(C(F)(F)F)C=C2)[13C]=C1,CC(C)(C)C1=CC=[15N]C(C2=[15N]C=CC(C(C)(C)C)=C2...,CC(C)(C)c1ccn2->[Ir+3]34(<-n5cc(C(F)(F)F)ccc5-...
1,2,FC1=CC(F)=C(C2=[15N]C=CC(C(C)(C)C)=C2)[13C]=C1,CC(C)(C)C1=CC=[15N]C(C2=[15N]C=CC(C(C)(C)C)=C2...,CC(C)(C)c1ccn2->[Ir+3]34(<-n5ccc(C(C)(C)C)cc5-...
2,3,FC1=CC(F)=C(C2=[15N]C=CC=C2)[13C]=C1,CC(C)(C)C1=CC=[15N]C(C2=[15N]C=CC(C(C)(C)C)=C2...,CC(C)(C)c1ccn2->[Ir+3]34(<-n5ccccc5-c5c(F)cc(F...
3,4,C1(C2=[15N]C=CC=C2)=CC=CC=[13C]1,CC(C)(C)C1=CC=[15N]C(C2=[15N]C=CC(C(C)(C)C)=C2...,CC(C)(C)c1ccn2->[Ir+3]34(<-n5ccccc5-c5cccc[c]5...
4,5,C1(C2=[15N]C=CC=C2)=CC=CC=[13C]1,COC1=CC=[15N]C(C2=[15N]C=CC(OC)=C2)=C1,COc1ccn2->[Ir+3]34(<-n5ccccc5-c5cccc[c]53)(<-n...
5,6,CC(C)(C)C1=CC=[13C]C(C2=[15N]C=CC(C(C)(C)C)=C2...,CC(C)(C)C1=CC=[15N]C(C2=[15N]C=CC(C(C)(C)C)=C2...,CC(C)(C)c1cc[c]2c(c1)-c1cc(C(C)(C)C)ccn1->[Ir+...
6,7,CC(C)(C)C1=CC=[13C]C(C2=[15N]C=CC(C(C)(C)C)=C2...,COC1=CC=[15N]C(C2=[15N]C=CC(OC)=C2)=C1,COc1ccn2->[Ir+3]34(<-n5ccc(C(C)(C)C)cc5-c5cc(C...


In [10]:
df.to_csv('ref_smiles.csv')